In [1]:
import pandas as pd
import numpy as np
import copy

# df_train = pd.read_csv('./data/raw/mnist_train.csv')
# df_test = pd.read_csv('./data/raw/mnist_train.csv')


In [2]:
# # train_data = np.array(df_train)
# # train_data.shape()
# df_train.head()


- 28 x 28 pixel image flattened to 784 column feature, each feature being one pixel intensity value.

In [3]:
# print(df_train.columns[:5])

In [4]:
# train_data = np.array(df_train)
# # m training examples and n features
# train_data.shape
# m, n = train_data.shape

In [5]:
# X_train = (train_data[:, 1:].T)/255
# y_train = train_data[:, 0]
# # X_train.shape  #((784, 60000)
# # y_train.shape  (60000,)

In [6]:
# y_train[1:5]
# print(y_train.dtype)

In [7]:
# COMPLETED
def parameter_init(L):
    parameters = {}

    for i in range(len(L) - 1):
        parameters["W" + str(i+1)] = np.random.randn(L[i+1], L[i]) * 0.05
        parameters["b" + str(i+1)] = np.zeros((L[i+1], 1))

    return parameters

In [8]:
# COMPLETED
def forward_linear(A, W, b):

    Z = np.dot(W, A) + b
    cache = (A, W, b)

    return Z, cache


In [9]:
# COMPLETED
def activation_fn(Z, act_fn):
    if act_fn == 'relu':
        return np.maximum(0, Z), Z
    
    elif act_fn == 'sigmoid':
        A = 1 / (1 + np.exp(-Z))
        return A, Z
    elif act_fn == 'softmax':
        Z_shifted = Z - np.max(Z, axis=0, keepdims=True)  # for numerical stability
        exp_Z = np.exp(Z_shifted)
        A = exp_Z / np.sum(exp_Z, axis=0, keepdims=True)
        return A, Z
    # ADD MORE ACTIVATION FUNCTIONS

In [10]:
# COMPLETED
def forward_activation(A_prev, W, b, activation):
    """ This Function calculates the forward prop for one layer and stores the activation and cache containin (A_prev, W, b)"""

     # capture both Z and also the A,W,b
    Z, linear_cache = forward_linear(A_prev, W, b)
    # activation_cache captures the Z value     
    A, activation_cache = activation_fn(Z, activation)
    
    
    cache = (linear_cache, activation_cache)
    # cache = [
    #          ((A, W, b), Z)
    #          ]

    return A, cache

In [11]:
# COMPLETED
def forward_propagation(parameters, X, activations):
    """ FWD prop """
    A = X
    L = len(parameters) // 2
    caches = []
    for l in range(1, L):
        A_prev = A
        A, cache = forward_activation(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], activation = activations[0])
        caches.append(cache)
        

    AL, cache  = forward_activation(A, parameters["W" + str(L)], parameters["b" + str(L)], activation = activations[1])
    caches.append(cache)

    #  caches  = [
    #             ((A, W, b), Z) ---> Layer 1
    #             ((A, W, b), Z) ---> layer 2
    #            .
    #            .
    #            .
    #            ((A, W, b), Z) ---> layer L
    #            ]
    
    return AL, caches


In [12]:
# COMPLETED
def compute_cost(AL, Y):
    """Computes Cost using Categorical Cross Entropy Loss Function

    Cost = (-1/m) * Sum for all examples -[Y*log(A)]
    """
    m = Y.shape[1]

    logar = np.multiply(Y, np.log(AL))
    
    cost = (np.sum(logar))/(-m)
    print(f"AL (example 3): {AL[:, 2]}")
    print(f"Y (example 3): {Y[:, 2]}")
    print(f"Sum of softmax probabilities for example 3: {np.sum(AL[:, 2])}")
    # cost = np.squeeze(cost) # to return just correct cost shape (float)

    # if iterations % 100 ==0 and print_cost == True:
    #     print(f"Iteration: {iterations}./n Cost : {cost}") 

    return cost

In [13]:
# CAN ADD MORE ACTIVATION DERIVATIVES
def act_derivatives(activation, cache):
    Z = cache
    if activation == 'sigmoid':
        s = 1 / (1 + np.exp(-Z))
        return s * (1 - s)
    
    elif activation == 'relu':
        dZ = np.array(Z > 0, dtype=float)
        return dZ
    elif activation == 'tanh':
        tanh = np.tanh(Z)
        dZ = 1 - tanh**2
        return dZ
    elif activation == 'leaky_relu':
        dZ = np.where(Z > 0, 1, 0.01)
        return dZ
    elif activation == 'softmax':
        # Softmax derivative is usually handled together with cross-entropy loss,
        # but if needed, return Jacobian (only for academic or special purposes)
        # This is a simplified softmax derivative per element (diagonal of Jacobian)
        exps = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        softmax = exps / np.sum(exps, axis=0, keepdims=True)
        dZ = softmax * (1 - softmax)  # Only valid if used element-wise
        return dZ

    # More Activation Derivates can be added.

In [14]:
# COMPLETED
def linear_backprop(dZ, cache):
    """ This Function computes the linear backward gradients which is the same irrespective of activation functions
      as Z = (W).(A_prev) + (b), '() -linear relationship' 
    """

    A_prev, W, b = cache
    # No of inputs from prev activation
    m = A_prev.shape[1]

    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis = 1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
     
    return dA_prev, dW, db

In [15]:
# COMPLETED
def activation_backprop(dA, cache, activation):

    linear_cache,activation_cache = cache

    # Passing activation cache which is Z
    dZ = np.multiply(dA, act_derivatives(activation, activation_cache))
    # Passing linear cache which is A, W, b
    dA_prev, dW, db = linear_backprop(dZ, linear_cache)

   
    # More activations can be defined
    
    return dA_prev, dW, db

In [16]:
# COMPLETE
def back_propagation(AL, Y, caches, activations):
    """ Calculates the gradient values for each parameter and returns a dictionary of them """

    gradients = {}
    L = len(caches)
    Y = Y.reshape(AL.shape)
    if activations[1] == "softmax":
        dAL = AL - Y  
        dZ = dAL                                           # for softmax + categorical crossentropy
    elif activations[1] == "sigmoid":
        dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))    # for sigmoid + binary crossentropy
    else:
        raise ValueError(f"Unsupported output activation: {activations[1]}")

    # # Derivative for the final output layer activation with CATEGORICAL CROSS ENTROPY LOSS
    # epsilon = 1e-12
    # AL = np.clip(AL, epsilon, 1. -epsilon)
    # dAL = -(np.divide(Y, AL))                        # Some formula depending on the Cost Function.

    # Index 0, 1, 2, 3... L so last layer is L-1                          
    last_cache = caches[L-1]
    # Output layer( L'th layer) gradients take dAL, cache containing (A_prev, W, b), activation of the ouput.
    # This is a single seperate computation because of unique dAL.
    linear_cache,activation_cache = last_cache
    # dA_prev_temp, dW_temp, db_temp = activation_backprop(dAL, last_cache, activation = activations[1])
    dA_prev_temp, dW_temp, db_temp = linear_backprop(dZ, linear_cache)

    gradients["dA" + str(L-1)] = dA_prev_temp
    gradients["dW" + str(L)] = dW_temp
    gradients["db" + str(L)] = db_temp


    # Now we have the dA_prev from the last layer and this can be used to compute all the gradients from (L-1)layer
    # in a single for loop because of the same activation.
    for l in reversed(range(L-1)):
        cache = caches[l]
        # From L-1 layer we have the same back prop till the first layer.
        # The reversed range gives L-2, L-3, ..... 2,1,0.
        dA_prev_temp, dW_temp, db_temp = activation_backprop(gradients["dA" + str(l+1)], cache=cache, activation = activations[0])
        gradients["dA" + str(l)] = dA_prev_temp
        gradients["dW" + str(l+1)] = dW_temp
        gradients["db" + str(l+1)] = db_temp

    # gradients = {
    #     "dW2": dW2,
    #     "db2": db2,
    #     "dW1": dW1,
    #     "db1": db1,
    # }
    return gradients

In [17]:
# COMPLETED
def gradient_descent(parameters, grads, learning_rate = 0.02):
    """ This function applies gradient descent, updates the weight and bias parameters using their respective gradients.
     
      
       Returns - parameters
    """

    parameters = copy.deepcopy(parameters)
    L = len(parameters) // 2
    for l in range(L):
        parameters["W" + str(l +1)] = parameters["W" + str(l +1)] - (learning_rate * grads["dW" + str(l+1)])
        parameters["b" + str(l +1)] = parameters["b" + str(l +1)] - (learning_rate * grads["db" + str(l+1)])
    
    # parameters["W1"] -= learning_rate * dW1
    # parameters["b1"] -= learning_rate * db1 

    return parameters

In [18]:
def train_NN(X, Y, n_h, activations, iterations = 900, learning_rate = 0.03, print_cost = False):
    """ This function takes the input X and output label Y and performs
            -forward propagation,
            -computes cost, and applies 
            -back propagation to calculate the gradients and 
            -updates the parameters 
            
    """

    L = [X.shape[0], *n_h]
    parameters = parameter_init(L)
    for iter in range(iterations):

        AL, caches = forward_propagation(parameters, X, activations = activations)
        # Print the cost for every 100 iterations.
        if iter % 100 == 0 and print_cost == True:
            cost = compute_cost(AL, Y)
            print("Cost after iteration {}: {}".format(iter, np.squeeze(cost))) 
        # Computing gradients
        grads = back_propagation(AL, Y, caches, activations)
        # Parameter update
        parameters = gradient_descent(parameters, grads, learning_rate)
        

    return parameters



- X_train.shape  #((784, 60000)
- y_train.shape  (60000,)
#### Now the shapes of input are correct and change the target label y to [1,0,0,0...] format for their respective value

In [19]:
def encode_y(Y):
    enc_y = np.zeros((Y.max() + 1, Y.size))
    # Placing 1 according to the label (Y value, index of size m)
    enc_y[Y, np.arange(Y.size)] = 1
    return enc_y

### Completed with encoding the target label now design the prediction and accuracy label

In [20]:
def predict_y(X: np.ndarray, parameters: dict[str, np.ndarray], activations) -> np.ndarray:

    AL,caches = forward_propagation(parameters, X, activations= activations)
    y_pred = np.argmax(AL, axis = 0)
    return y_pred

In [21]:
def accuracy(true_y, predicted_y):
    return np.mean(true_y == predicted_y)
    # print("ACCURACY :", accuracy)
    # print(f"Your Neural Networks Prediction is {predicted_y}")
    # print(f"The actual Label of the given input is {true_y}")
    


In [24]:
def main():
    df_train = pd.read_csv('./data/raw/mnist/mnist_train.csv')
    df_test = pd.read_csv('./data/raw/mnist/mnist_test.csv')
    train_data = np.array(df_train)
    test_data = np.array(df_test)

    n_h = [32,16,10]
    activations = ['relu', 'softmax']
    # m training examples and n features
    train_data.shape
    m, n = train_data.shape
    X_train = (train_data[:, 1:].T)/255
    y_train = train_data[:, 0]
    # X_train.shape  #((784, 60000)
    # y_train.shape  (60000,)
    
    X_test = (test_data[:, 1:].T)/255
    y_test = test_data[:, 0]

 
    y_train_enc = encode_y(y_train)
    params = train_NN(X_train, y_train_enc, n_h, activations, iterations = 800, learning_rate = 0.1, print_cost = True)

    train_predictions = predict_y(X_train, params, activations=activations)
    train_accuracy = accuracy(y_train, train_predictions)
    print(f"The accuracy of the NN in Training : {train_accuracy:.4f}\n")

    
    test_predictions = predict_y(X_test, params, activations=activations)
    test_accuracy = accuracy(y_test, test_predictions)
    print(f"The accuracy of the NN in Test : {test_accuracy:.4f}")
    

if __name__ == '__main__':
    main()




    

    

AL (example 3): [0.10096703 0.10025725 0.10204066 0.09860601 0.09913204 0.10088912
 0.09949888 0.10057553 0.09902804 0.09900543]
Y (example 3): [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
Sum of softmax probabilities for example 3: 1.0
Cost after iteration 0: 2.3036966048972913
AL (example 3): [0.09919359 0.10356958 0.10042135 0.09832545 0.10082086 0.09216396
 0.09844728 0.10932095 0.09717127 0.10056571]
Y (example 3): [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
Sum of softmax probabilities for example 3: 1.0
Cost after iteration 100: 2.2400459406317657
AL (example 3): [0.01245518 0.04164405 0.06827169 0.0282301  0.32512956 0.03661728
 0.13096957 0.08283245 0.07386674 0.19998337]
Y (example 3): [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
Sum of softmax probabilities for example 3: 1.0
Cost after iteration 200: 1.206882986085008
AL (example 3): [0.00366474 0.00112528 0.0117322  0.00243467 0.58109061 0.02616084
 0.18582079 0.00662527 0.05967845 0.12166714]
Y (example 3): [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
Sum of softmax proba